In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# Libraries
import os
import pickle

from src.models import train_model, test_model, predict_model

In [19]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [20]:
EXPERIMENT = 'mh07'
MODEL = 'model_A1'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/{MODEL}/n1000/mutate_m1_20_prand/'
# Models
model_template = f'./models/templates/{MODEL}.pl'

Training

In [21]:
train_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_DATA, 
    model_template = model_template, 
    training = 0.8, 
    max_iter = 3, 
    evidence = ['occupancy'],
    facts = ['activity', 'fold_change'],
    suffix = ''
)

Splitting data...
Starting LFI...
problog lfi ./models/mh07/model.pl ./models/mh07/evidence.pl -O ./models/mh07/model_trained.pl --normalize -k ddnnf -v -n 3
[INFO] Output level: INFO
[INFO] Number of examples: 800
[INFO] Compile time: 9.224871
[INFO] Weights to learn: [t(0.33,P,_fc,_dM1,_dM2,_dM3)::occupancy(P,S,inc), t(0.33,P,_fc,_dM1,_dM2,_dM3)::occupancy(P,S,dec), t(0.33,P,_fc,_dM1,_dM2,_dM3)::occupancy(P,S,baseline)]
[INFO] Bodies: [lfi_body(0,t(P,_fc,_dM1,_dM2,_dM3),P,S,inc), lfi_body(1,t(P,_fc,_dM1,_dM2,_dM3),P,S,dec), lfi_body(2,t(P,_fc,_dM1,_dM2,_dM3),P,S,baseline)]
[INFO] Parents: [lfi_par(0,t(P,_fc,_dM1,_dM2,_dM3),P,S,inc), lfi_par(1,t(P,_fc,_dM1,_dM2,_dM3),P,S,dec), lfi_par(2,t(P,_fc,_dM1,_dM2,_dM3),P,S,baseline)]
[INFO] Initial weights: [0.33, 0.33, 0.33]
[INFO] 
[INFO] Iteration 1
[INFO] Weights after iteration 1: [{t(p0,inc,baseline,inc,inc): 1.0, t(p0,baseline,dec,inc,inc): 0.5002704202347918, t(p0,dec,baseline,inc,dec): 0.10029067629648442, t(p0,dec,dec,baseline,baseli

Testing

In [22]:
# Testing
model_trained = f'{EXP_DIR}model_trained.pl'
test = ['training', 'testing']
PATH_TO_TRUTH = f'data/processed/synthetic_data/{MODEL}/n1000/'
results = {}

In [ ]:
for test_on in test:
    PATH_TO_TESTING = f'{EXP_DIR}{test_on}/'
    compare_to = [test_on, 'truth']
    for compare in compare_to:
        if compare == 'truth':
            path_to_truth = PATH_TO_TRUTH
        else: 
            path_to_truth = PATH_TO_TESTING

        test_results = test_model.main(
            experiment = EXPERIMENT, 
            path_to_data = PATH_TO_TESTING, 
            path_to_truth = path_to_truth, 
            model_trained = model_trained, 
            targets = ['occupancy', 'modification'], 
            test_on = test_on, compare = compare)
        results[f'{test_on}_vs_{compare}'] = test_results

In [24]:
PATH_TO_TESTING = f'{EXP_DIR}testing/'
predictions = predict_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_TESTING, 
    model_trained = model_trained, 
    targets = ['occupancy', 'modification'], 
    mode = 'inference', infer_from='facts', 
    suffix = '_testing')
results['predictions'] = predictions

Inferring data (s540)...
problog ./models/mh07/make_predictions.pl -o ./models/mh07/predictions/inferred_example.txt
Finished inference (s540)...
Inferring data (s212)...
problog ./models/mh07/make_predictions.pl -o ./models/mh07/predictions/inferred_example.txt
Finished inference (s212)...
Inferring data (s10)...
problog ./models/mh07/make_predictions.pl -o ./models/mh07/predictions/inferred_example.txt
Finished inference (s10)...
Inferring data (s281)...
problog ./models/mh07/make_predictions.pl -o ./models/mh07/predictions/inferred_example.txt
Finished inference (s281)...
Inferring data (s943)...
problog ./models/mh07/make_predictions.pl -o ./models/mh07/predictions/inferred_example.txt
Finished inference (s943)...
Inferring data (s763)...
problog ./models/mh07/make_predictions.pl -o ./models/mh07/predictions/inferred_example.txt
Finished inference (s763)...
Inferring data (s588)...
problog ./models/mh07/make_predictions.pl -o ./models/mh07/predictions/inferred_example.txt
Finished 

In [25]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)